<a href="https://colab.research.google.com/github/MatthewYancey/GANime/blob/master/src/scoring_lpips.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scoring
This notebook takes the output test restults from the models and calculates their frechet inception distance

In [ ]:
!git clone https://github.com/richzhang/PerceptualSimilarity
!pip install -r PerceptualSimilarity/requirements.txt

In [8]:
import shutil
import os
import glob
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [14]:
test_output_ce = '/content/gdrive/MyDrive/repos/GANime/data_out/test_output/ce.zip'
test_output_gal = '/content/gdrive/MyDrive/repos/GANime/data_out/test_output/gal.zip'
test_output_ffc = '/content/gdrive/MyDrive/repos/GANime/data_out/test_output/ffc.zip'
test_dataset = '/content/gdrive/MyDrive/repos/GANime/data_out/pokemon/test.zip'

temp_dir_1 = '/content/temp_folder_1/'
temp_dir_2 = '/content/temp_folder_2/'

In [16]:
def get_score(input_zip):
    print(input_zip)

    print('Copying the zip file over')
    if os.path.isdir(temp_dir_1):
        shutil.rmtree(temp_dir_1)

    print('Unpacking zip file')
    shutil.copy(input_zip, '/content/')
    shutil.unpack_archive(f'/content/{os.path.basename(input_zip)}', temp_dir_1)

    # unpacks the test imagse if they are not already there
    if os.path.isdir(temp_dir_2) == False:
        shutil.copy(test_dataset, '/content/')
        shutil.unpack_archive(f'/content/{os.path.basename(test_dataset)}', temp_dir_2)

    # rename the files
    temp_1_list = glob.glob(f'{temp_dir_1}*')
    temp_2_list = glob.glob(f'{temp_dir_2}*')

    temp_1_list.sort()
    temp_2_list.sort()

    print('Renaming files')
    # have to copy and rename the files
    for i in range(len(temp_1_list)):
        new_file_name = os.path.basename(temp_1_list[i])
        new_file_name = new_file_name.split('_')[1]
        n_digits = len(new_file_name) - 4
        new_file_name = '0' * (5 - n_digits) + new_file_name

        # rename the file
        os.rename(temp_1_list[i], f'{temp_dir_1}{new_file_name}')    

    print('Calculating score')
    # have to copy and rename the files
    for i in range(len(temp_2_list)):
        new_file_name = str(i) + '.jpg'
        n_digits = len(new_file_name) - 4
        new_file_name = '0' * (5 - n_digits) + new_file_name

        # rename the file
        os.rename(temp_2_list[i], f'{temp_dir_2}{new_file_name}')


    os.system('python PerceptualSimilarity/lpips_2dirs.py -d0 temp_folder_1 -d1 temp_folder_2 -o PerceptualSimilarity/imgs/example_dists.txt --use_gpu')

    df = pd.read_csv('/content/PerceptualSimilarity/imgs/example_dists.txt', sep=' ', header=None)
    print(df.iloc[:, 1].mean())


In [18]:
for z in [test_output_ce, test_output_gal, test_output_ffc]:
    get_score(z)

/content/gdrive/MyDrive/repos/GANime/data_out/test_output/ce.zip
0.14625632196969704
/content/gdrive/MyDrive/repos/GANime/data_out/test_output/gal.zip
0.13956444318181815
/content/gdrive/MyDrive/repos/GANime/data_out/test_output/ffc.zip
0.1371621060606061
